In [ ]:
%matplotlib inline


Document Clustering example (Python)
------------------------------------

An example of clustering using Python API



In [ ]:
import pandas as pd
from IPython.display import display
from freediscovery.text import FeatureVectorizer
from freediscovery.cluster import Clustering
from freediscovery.utils import _silent
from time import time

pd.options.display.float_format = '{:,.3f}'.format

dataset_name = "treclegal09_2k_subset"

data_dir = "../freediscovery_shared/{}".format(dataset_name)
examples_to_server_path = "../" # relative path between this file and the FreeDiscovery source folder

BASE_URL = "http://localhost:5001/api/v0"  # FreeDiscovery server URL


# # 1. Feature extraction (non hashed)

n_features = 30000
cache_dir = '/tmp/'

fe = FeatureVectorizer(cache_dir=cache_dir)
uuid = fe.preprocess("../"+data_dir+'/data',
                     n_features=n_features, use_hashing=False,
                     use_idf=True, stop_words='english')
uuid, filenames = fe.transform()


# # 2. Document Clustering (LSI + K-Means)

cat = Clustering(cache_dir=cache_dir, dsid=uuid)

n_clusters = 10
n_top_words = 6
lsi_components = 50


def repr_clustering(_labels, _terms):
    out = []
    for ridx, row in enumerate(_terms):
        out.append({'cluster_names': row, 'N_documents': (_labels == ridx).sum()})
    out = pd.DataFrame(out).sort_values('N_documents', ascending=False)
    return out


t0 = time()
with _silent('stderr'): # ignore some deprecation warnings
    labels, tree  = cat.k_means(n_clusters, lsi_components=lsi_components)
    terms = cat.compute_labels(n_top_words=n_top_words)
t1 = time()

print('    .. computed in {:.1f}s'.format(t1 - t0))
display(repr_clustering(labels, terms))


# # 3. Document Clustering (LSI + Ward Hierarchical Clustering)

t0 = time()
with _silent('stderr'): # ignore some deprecation warnings
    labels, tree = cat.ward_hc(n_clusters,
                               lsi_components=lsi_components,
                               n_neighbors=5   # this is the connectivity constraint
                               )
    terms = cat.compute_labels(n_top_words=n_top_words)
t1 = time()

print('    .. computed in {:.1f}s'.format(t1 - t0))
display(repr_clustering(labels, terms))